# Введение в регулярные выражение в python

Для тех, кто в танке) Работа с регулярными выражениями в питона осуществляется с помощью модуля re. Регулярные выражения служат, как правило, двум целям. Первое. Проверить, что строка удовлетворяет заданному шаблону. Второе. Найти все вхождения шаблона в строку. За первое отвечает match, за второе - search или find. Еще есть опция замены одного шаблона на строку или другой шаблон. За это отвечает sub и subn. Писанные сущности могут быть как функциями модуля re, так и методами скомпилированного  регулярного выражения. Но об этом подробно далее. 

# Основы

In [6]:
import re

Рассматривать примеры пока будем на основании функции match. Функция служит для проверки того, что строка соответствует заданному шаблону. Пример. Пусть есть шаблон 

In [7]:
pattern = r'abc|cde'

Ему удовлетворяют все строки, которые начинаются на abc или на cde. Давайте это проверим

In [8]:
def check_match(pattern, line, flags=0):
    m = re.match(pattern, line, flags)
    if not m:
        print '"' + line + '"' + " does not match pattern " + '"' + pattern + '"'
        return
    print '"' + line + '"' + " matches patern " + '"' + pattern + '"'
    print '"' + m.group() + '"' + " provides this match"

In [9]:
lines = [
    'a',
    'abc',
    'abcdelkj',
    'cde',
    ' 123 sd'
]
for line in lines:
    check_match(pattern, line)

"a" does not match pattern "abc|cde"
"abc" matches patern "abc|cde"
"abc" provides this match
"abcdelkj" matches patern "abc|cde"
"abc" provides this match
"cde" matches patern "abc|cde"
"cde" provides this match
" 123 sd" does not match pattern "abc|cde"


Что тут нужно заметить. Первое. Если начало строки удовлетворяет шаблону, то вся строка удовлетворяет шаблону. Второе. Если строка не сооветствует шаблону (не матчится), то re.match возвращает None. Третье. Если строка шаблону удовлетворяет, то можно увидеть, по какому набору символов удалось сопоставить (заматчить) шаблон и строку. Для этого достаточно вызвать метод group() у объекта, который возвращает функция re.match().

# Синтаксис регулярных выражений. 

Простейший случай мы разобрали выше. Подряд идущая группа символов описывает точно то, с чем должно совпадать начало строки. Знак вертикальной черты "|", также называемый оператором pipe или конвеером, означает в регулярных выражениях логическое ИЛИ.

Символ точки "." означает любой символ, кроме знака перевода строки "\n". С помощью флагов компиляции можно указать, чтобы точка обозначала вообще любой символ, даже символ перевода строки, но об этом чуть позже. Пример.

In [10]:
pi_pattern = '3.14' # здесь точка означает любой символ, кроме символа перевода строки
exact_pi_pattern = '3\.14' # здесь в шаблоне регулярного выражения \. означает десятичную точку

check_match(pi_pattern, '3.14')
check_match(exact_pi_pattern, '3.14')
check_match(pi_pattern, '3014')
check_match(exact_pi_pattern, '3014')

"3.14" matches patern "3.14"
"3.14" provides this match
"3.14" matches patern "3\.14"
"3.14" provides this match
"3014" matches patern "3.14"
"3014" provides this match
"3014" does not match pattern "3\.14"


Квадратные скобки позволяют задвать набор символов, которые должны стоять на заданном месте. Например регулярное выражение [ab][de] обозначает строки, которые начинаются на ad, или на ae, или на bd, или на be

In [11]:
pattern = '[cr][23][dp][o2]'
check_match(pattern, 'r2d2')
check_match(pattern, 'c3po')
check_match(pattern, 'ololo')

"r2d2" matches patern "[cr][23][dp][o2]"
"r2d2" provides this match
"c3po" matches patern "[cr][23][dp][o2]"
"c3po" provides this match
"ololo" does not match pattern "[cr][23][dp][o2]"


Указав в начале квадратных скобок знак домика "^", можно поменять смысл квадратных скобок на противоположный - теперь они будут обозначать любой знак, который не встречается в квадратных скобках после знака домика. 

In [12]:
pattern = '[^cr][^23][^dp][^o2]'
check_match(pattern, 'r2d2')
check_match(pattern, 'c3po')
check_match(pattern, 'ololo')

"r2d2" does not match pattern "[^cr][^23][^dp][^o2]"
"c3po" does not match pattern "[^cr][^23][^dp][^o2]"
"ololo" matches patern "[^cr][^23][^dp][^o2]"
"olol" provides this match


# Диапазоны символов (range)

В квадратных скобках можно указывать не только наборы символов, но и диапазоны. Например, мы хотим проверить, что строка начинается с четырех цифр, каждая из которых - от двух до семи (уж не знаю, за чем нам понадобились таки странные числа, но все же давайте напряжем воображение). Вместо того, чтобы писать 4 раза [234567], можно 4 раза написать [2-7]

In [13]:
pattern = '[2-7][2-7][2-7][2-7]'
check_match(pattern, 'r2d2')
check_match(pattern, 'c3po')
check_match(pattern, '2534 sdf')

"r2d2" does not match pattern "[2-7][2-7][2-7][2-7]"
"c3po" does not match pattern "[2-7][2-7][2-7][2-7]"
"2534 sdf" matches patern "[2-7][2-7][2-7][2-7]"
"2534" provides this match


# Символы повторения

Солгласитесь, что странно 4 раза в примере выше писать один и тот же код "[2-7]". И действительно, есть куча способов сказать, что мы не один раз хотим увидеть цифру от двух до семи, а несколько. Для этого есть специальные символы повторения. Символ * в регулярном выражении означает, что символ может встречаться ноль или более раз, символ + означает, что символ может встречаться один или более раз, запись {4} означает, что символ или группа символов встречается 4 раза. Четверку можно заменить на любое натуральное число. Примеры в студию

In [14]:
check_match('[2-7]*', ' 2345 lkj')
check_match('[2-7]+', 'c3po')
check_match('[2-7]{4}', '2534 sdf')
check_match('[2-7]{5}', '2534 sdf')

" 2345 lkj" matches patern "[2-7]*"
"" provides this match
"c3po" does not match pattern "[2-7]+"
"2534 sdf" matches patern "[2-7]{4}"
"2534" provides this match
"2534 sdf" does not match pattern "[2-7]{5}"


# Специальные символы

Для обозначения цифр, знаков пробелов, цифро-буквенных символов применяются специальные символы.

\d обозначает любую из цифр. Эквивалентен [0-9]

\D обозначает любой символ, кроме цифры. Эквивалентен [^0-9]

\w обозначает любой алфавитно-цифровой символ. Если быть совсем точным, то он эквивалентен [0-9a-zA-Z_]

\W обозначает любой символ, который не попадает под описание  \w из прошлой строк 

\s обозначает любой пробельный символ, например пробел, перевод строки, табуляция и прочее. Эквивалентен [\n\t\r\v\f]

\S обозначет любой символ, не попадающий под описание \s

^ обозначает начало строки

$ обозначает конец строки

\b обозначает границу слова - не важно, начало или конец

\N, где N - это натуральное число, обозначает номер группы. Об этом далее.

Примеры с началом и концом строк, пробелами

In [22]:
patterns = [
    r'^The',
    r'\bThe',
    r'\Bthe',
    r'(\w+\s+){2}\w*the[\w\s]*' # убранное в скобки выражение называется группой, {2} означает, что 
                                # выражение перед {2} должно встретиться два раза. Вместо 2 может быть
                                # любое натуральное число. Можно также здавать не точное число раз, а диапазон
                                # {2,5} говорит о том, что символ или группа перед {2,5} должны встретиться
                                # не меньше двух и не более 5 раз
]
line = 'The best inthe world Hello!'
for p in patterns:
    check_match(p, line)

"The best inthe world Hello!" matches patern "^The"
"The" provides this match
"The best inthe world Hello!" matches patern "\bThe"
"The" provides this match
"The best inthe world Hello!" does not match pattern "\Bthe"
"The best inthe world Hello!" matches patern "(\w+\s+){2}\w*the[\w\s]*"
"The best inthe world Hello" provides this match


# Подробнее про функции и методы классов re

До сих пор я упорно скрывал синтаксис команд регулярных выражений, завернув их в функцию check_match.

Пришло время рассказать подробнее.

В простейшем случае проверка матчинга строки шаблону происходит так

In [23]:
line = 'olololosl'
pattern = r'\w+'

m = re.match(pattern, line)
if m:
    print m.group()

olololosl


re.match возвращает объект регулярного выражения. Если заматчить строку шаблону не получилось, то вернется None, поэтому, чтобы не выкидывались исключения, обычно проверяют, что вернулось - для этого в данном примере используется if.

метод group объекта регулярного выражения возвращает стоку, которую удалось сопоставить регулярному выражению.

Все, что в регулярном выражении заключено в круглые скобки, считается так называемой группой, и к элементам группы можно обращаться через метод group, в случае удачного матча строки и регулярного выражения

In [25]:
line = 'abc-123'
pattern = r'(\w+)-(\d+)'

m = re.match(pattern, line)
if m:
    print m.group(1)
    print m.group(2)
    print m.groups() # еще есть метод groups, который сразу вернет содержимое групп в виде кортежа

abc
123
('abc', '123')


Согласитесь, полезная штука. Позволит доставать части регулярно выражения задешево. Без дополнительных плат на то, чтобы разделять то, что нашлось с помощью регулярного выражения.

Но с помощью регулярных выражений можно не только проверять, подходит ли строка под шаблон, но и находить шаблоны в строке. Для этого используется функция re.search или одноименны метод объекта скомпилированного регулярного выражения (у объектов скомпилированных регулярных выражений есть и метод match тоже; об этом чуть далее будет отдельно). С помощью этого метода можно проверить вхождения шаблона в строку, шаблон может входить в строку в любом месте, не только в начале

Простейший пример

In [29]:
line = 'abcdef 098'
patterns = [
    r'\w',
    r'\w+',
    r'\b09'
]
for p in patterns:
    s = re.search(p, line)
    print s.group()

a
abcdef
09
